## StepbyStep搭建分词工具

### 方法 1  穷举法搭建中文分词

给定词典=[自然，语言，处理，是，机器，学习，机器学习，的，一个，应用，场景，自然语言处理， 自然语言，应用场景， 语言处理]
另外给定unigram概率：p(自然语言处理)=0.25, p(机器学习)=0.15, p(应用场景)=0.05, p(机器学习)=0.1, p(的)=0.2, p(一个)=0.1, p(是)=0.15

#### Step 1: 对于给定句子：”自然语言处理是机器学习的一个应用场景“, 找出所有可能的分词方式
- [自然，语言，处理，是，机器，学习，的，一个，应用，场景]
- [自然语言处理，是，机器学习，的，一个，应用场景]
- [自然语言，处理，是，机器，学习，的，一个，应用，场景]
- [自然，语言处理，是，机器，学习，的，一个，应用场景]
.......


#### Step 2: 计算出每一个分词之后句子的概率
- p(自然，语言，处理，是，机器，学习，的，一个，应用，场景)= -log p(自然)-log p(语言)-log p(处理)-log p(是)-log p(机器)-log p(学习)-log p(的)-log p(一个)-log p(应用)-log p(场景)
- p(自然语言处理，是，机器学习，的，一个，应用场景)=-log p(自然语言处理)-log p(是)-log p(机器学习)-log p(的)-log p(一个)-log p(应用场景)
- p(自然语言，处理，是，机器，学习，的，一个，应用，场景)=-log p(自然语言)-log p(处理)-log p(是)-log p(机器)-log p(学习)-log p(的)-log p(一个)-log p(应用)-log p(场景)
- p(自然，语言处理，是，机器，学习，的，一个，应用场景)=-log p(自然)-log p(语言处理)-log p(是)-log p(机器)-log p(学习)-log p(的)-log p(一个)-log p(应用场景)
.....

#### Step 3: 返回第二步中概率最大的分词结果

In [1]:
word_prob = {"上海":0.03,"的":0.08,"天":0.005,"气":0.005,"天气":0.06,"真":0.04,"好":0.05,"真好":0.04,"啊":0.03, 
             "今":0.01,"今天":0.07,"电视":0.06,"电视剧":0.06,"有":0.05,"很":0.04,"趣":0.06,"有趣":0.035,"电":0.01,
             "视":0.005,"经常":0.08,"意见":0.08,"意":0.01,"见":0.005,"有意见":0.02,"分歧":0.04,"分":0.02, "歧":0.005}
dic_words = set(word_prob.keys())
# 如果有其他的字典可以存放到dic_words里
print (sum(word_prob.values()))

1.0000000000000002


In [2]:
def seg_helper(s, wordDict, memo):
    if s in memo: return memo[s]
    if not s: return []

    res = []
    for i in range(len(s)):
        if s[:i+1] not in wordDict:
            continue
        if i == len(s)-1:
            res.append([s[:i+1]])
        else:
            resultOfTheRest = seg_helper(s[i+1:], wordDict, memo)
            for item in resultOfTheRest:
                item = [s[:i+1]] + item
                res.append(item)
    memo[s] = res
    return res

In [3]:
from math import log10

def seg_prob(segment, word_prob):
    best_score = float("inf")
    best_segment = []
    for segment in segments:
        prob = 0
        for word in segment:
            if word in word_prob.keys():
                prob += -log10(word_prob[word])
            else:
                prob += -log10(0.00001)
#         prob = prob/len(segment)
        if best_score >= prob:
            best_score = prob
            best_segment = segment
    return best_score, best_segment
        
                

In [4]:
def NaiveSegmentor(input_str):
    """
    1. 对于输入字符串做分词，并返回所有可行的分词之后的结果。
    2. 针对于每一个返回结果，计算句子的概率
    3. 返回概率最高的分词
    
    input_str: 输入字符串   输入格式：“上海今天的天气真好”
    best_segment: 最好的分词结果  输出格式：["上海"，"今天"，"的"，"天气","真好"]
    """

    # 第一步： 计算所有可能的分词结果，要保证每个分完的词存在于词典里，这个结果有可能会非常多。 
    segments = []  # 存储所有分词的结果。如果次字符串不可能被完全切分，则返回空列表(list)
                   # 格式为：segments = [['上海', '的', '天', '气', '真', '好', '啊'], ['上海', '的', '天', '气', '真好', '啊'],...]
    segments = seg_helper(input_str, dic_words, {})
    # 第二步：循环所有的分词结果，并计算出概率最高的分词结果，并返回
    best_segment = []
    best_score = float("inf")
    for seg in segments:
        prob = 0
        for word in seg:
            if word in word_prob.keys():
                prob += -log10(word_prob[word])
            else:
                prob += -log10(0.00001)
#         prob = prob/len(seg)
        if best_score >= prob:
            best_score = prob
            best_segment = seg
    
    return best_segment      

In [5]:
# 测试
print(NaiveSegmentor("上海的天气真好啊"))
print(NaiveSegmentor("今天的电视剧很有趣"))
print(NaiveSegmentor("经常有意见分歧"))

['上海', '的', '天气', '真好', '啊']
['今天', '的', '电视剧', '很', '有趣']
['经常', '有意见', '分歧']


### 方法 2  用维特比算法来优化上述流程
#### 1.代码展示

In [6]:
def graph(input_str):
    graph = []
    # 定义为以这个字为结尾的所有单词
    for i in range(1,len(input_str)+1,1):
        path_i = {i-1:-log10(0.00001)}
        for j in range(i-1, -1, -1):
            word = input_str[j:i]
            if word in dic_words:
                if word in word_prob.keys():
                    path_i[j] = -log10(word_prob[word])
                else:
                    path_i[j] = -log10(0.00001)
        graph.append(path_i)
    return graph

def viterbi_path_direct(input_str, graph):
    memo_path = {}
    memo_score = {}
    for i in range(len(input_str)):
        best_score = float('inf')
        best_path = []
        best_seg = []
        for start_word in graph[i].keys():
            #calculate score
            if start_word == 0:
                score = graph[i][start_word]
#                 path = [start_word] + [i]
                path = [input_str[:i+1]]
                
            elif start_word == i:
                score = memo_score[start_word-1] + graph[i][i]
#                 path = memo_path[start_word-1] + [i]
                path = memo_path[start_word-1] +  [input_str[start_word:i+1]]
                
            elif start_word != 0:
                score = memo_score[start_word-1] + graph[i][start_word]
#                 path = memo_path[start_word-1] + [i]
                path = memo_path[start_word-1] + [input_str[start_word:i+1]]
                
            
            # compare and save
            if score <= best_score:
                best_score = score
                best_path= path
                
        memo_path[i] = best_path
        memo_score[i] = best_score
    
    return memo_path[len(input_str)-1]

def ViterbiSegmentor(input_str):
    graph_input = graph(input_str)
    return viterbi_path_direct(input_str, graph_input)
# 测试
print(ViterbiSegmentor("上海的天气真好啊"))
print(ViterbiSegmentor("今天的电视剧很有趣"))
print(ViterbiSegmentor("经常有意见分歧"))

['上海', '的', '天气', '真好', '啊']
['今天', '的', '电视剧', '很', '有趣']
['经常', '有意见', '分歧']


#### 2.分步骤理解以上过程
    1. 基于输入字符串，词典，以及给定的unigram概率来创建DAG(有向图）。
    2. 编写维特比算法来寻找最优的PATH
    3. 返回分词结果

In [7]:
def viterbi(input_str, graph):
    memo_path = {}
    memo_score = {}
    for i in range(len(input_str)):
        best_score = float('inf')
        best_path = []
        best_seg = []
        for start_word in graph[i].keys():
            #calculate score
            if start_word == 0:
                score = graph[i][start_word]
                path = [start_word] + [i]
#                 path = [input_str[:i+1]]
                
            elif start_word == i:
                score = memo_score[start_word-1] + graph[i][i]
                path = memo_path[start_word-1] + [i]
#                 path = memo_path[start_word-1] +  [input_str[start_word:i+1]]
                
            elif start_word != 0:
                score = memo_score[start_word-1] + graph[i][start_word]
                path = memo_path[start_word-1] + [i]
#                 path = memo_path[start_word-1] + [input_str[start_word:i+1]]
                
            
            # compare and save
            if score <= best_score:
                best_score = score
                best_path= path
                
        memo_path[i] = best_path
        memo_score[i] = best_score
    
    return memo_path[len(input_str)-1]


def ViterbiSegmentor(input_str):
    """
    1. 基于输入字符串，词典，以及给定的unigram概率来创建DAG(有向图）。
    2. 编写维特比算法来寻找最优的PATH
    3. 返回分词结果
    
    input_str: 输入字符串   输入格式：“上海今天的天气真好”
    best_segment: 最好的分词结果  输出格式：["上海"，"今天"，"的"，"天气","真好"]
    """
    
    # 第一步：
    # 根据词典，词典概率，和输入的句子，以及给定的unigram概率来创建带权重的有向图
    # 有向图的每一条边对应为一个单词的概率，
    # 这些概率在 word_prob，如果不在word_prob里的单词但在
    # 词典里存在的，统一用概率值0.00001。
    # 不一定有只有一种方式来存储这种结构。 
    graph = []
    # 定义为以这个字为结尾的所有单词
    for i in range(1,len(input_str)+1,1):
        path_i = {i-1:-log10(0.00001)}
        for j in range(i-1, -1, -1):
            word = input_str[j:i]
            if word in dic_words:
                if word in word_prob.keys():
                    path_i[j] = -log10(word_prob[word])
                else:
                    path_i[j] = -log10(0.00001)
        graph.append(path_i)
           
    
    
    # 第二步：
    # 利用维特比算法来找出最好的路径， 使得P(sentence)最大 。
    # 使用negative log sum:  -log(w1)-log(w2)-...
    # 所以返回-log P(sentence)最小的PATH

    # 太长了于是写了函数，注意这里的维特比函数和上面展示的有所区别
    
    path = viterbi(input_str, graph) 
    
    # 第三步： 根据最好路径, 返回最好的分词
    start = 0
    best_segment = []
    for idx in path[1:]:
        best_segment.append(input_str[start:idx+1])
        start = idx+1
    
    return best_segment

In [8]:
#测试
print(ViterbiSegmentor("上海的天气真好啊"))
print(ViterbiSegmentor("今天的电视剧很有趣"))
print(ViterbiSegmentor("经常有意见分歧"))
print(ViterbiSegmentor("我们经常有意见分歧")) # 因为 我们 没有在字典里，因此直接分开了

['上海', '的', '天气', '真好', '啊']
['今天', '的', '电视剧', '很', '有趣']
['经常', '有意见', '分歧']
['我', '们', '经常', '有意见', '分歧']


In [9]:
# todo
# 需要其他字的话可以利用dic_words变量存储字典
# 就是一个小demo哈哈
# 骚操作来了，一般来说……

# 绝招放在最后，其实最简单的办法就是

In [10]:
import jieba

#### 然后

In [11]:
text = "故天之道利而不害圣人之道为而弗争"
text2 = "道可道也非恒道也名可名也非恒名也"
text3 = "有名万物之始也无名万物之母也"
# jieba.cut直接得到generator形式的分词结果
seg = jieba.cut(text)  
print(' '.join(seg)) 
print(' '.join(jieba.cut(text2)))
print(' '.join(jieba.cut(text3)))
# ……似乎对古文的分词不太友好
text4 = "经常有意见分歧"
print(' '.join(jieba.cut(text4)))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/2x/836bc66165lc4dm4r05cmh7c0000gn/T/jieba.cache
Loading model cost 0.780 seconds.
Prefix dict has been built succesfully.


故天 之道 利而 不害 圣人 之道 为 而 弗争
道 可道 也 非恒道 也 名 可名 也 非恒名 也
有名 万物 之始 也 无名 万物 之母 也
经常 有 意见分歧
